# Audioset

# Setup

In [1]:
from tqdm import tqdm
import tensorflow as tf
from utils import vggish_slim
from utils import vggish_input
from utils import vggish_params
from utils import vggish_postprocess

In [9]:
import numpy as np
import pandas as pd

items = pd.read_csv("../Datasets/ml-100k/Text/items.csv")

In [10]:
items

,Unnamed: 0,Unnamed: 0.1,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,...,Thriller,War,Western,Summary,Cast,Director,Rating,Runtime,No. of ratings,YT-Trailer ID
0,0,0,1,Toy Story (1995),01-Jan-1995,NaN,https://www.imdb.com/title/tt0114709/,0,0,0,...,0,0,0,A little boy named Andy loves to be in his roo...,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,8.3,81.0,930K,mGXHH9iAfLA
1,1,1,2,GoldenEye (1995),01-Jan-1995,NaN,https://www.imdb.com/title/tt0113189/,0,1,1,...,1,0,0,When a deadly satellite weapon system falls in...,Pierce Brosnan|Sean Bean|Izabella Scorupco|Fam...,Martin Campbell,7.2,130.0,248K,IWFtzQOX_nY
2,2,2,3,Four Rooms (1995),01-Jan-1995,NaN,https://www.imdb.com/title/tt0113101/,0,0,0,...,1,0,0,This movie features the collaborative director...,Sammi Davis|Amanda De Cadenet|Valeria Golino|M...,Allison Anders,6.8,98.0,102K,hugpRyTc1fE
3,3,3,4,Get Shorty (1995),01-Jan-1995,NaN,https://www.imdb.com/title/tt0113161/,0,1,0,...,0,0,0,"Some guys get all the luck, whether they like ...",John Travolta|Gene Hackman|Rene Russo|Danny De...,Barry Sonnenfeld,6.9,105.0,81K,yvsnu3crV8g
4,4,4,5,Copycat (1995),01-Jan-1995,NaN,https://www.imdb.com/title/tt0112722/,0,0,0,...,1,0,0,"In San Francisco, the criminal psychologist He...",Sigourney Weaver|Holly Hunter|Dermot Mulroney|...,Jon Amiel,6.6,123.0,57K,gMrMLbxEt5Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1677,1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,https://www.imdb.com/title/tt0119711/,0,0,0,...,0,0,0,A slow and poignant story of love and patience...,Aleksei Ananishnov|Gudrun Geyer,Aleksandr Sokurov,7.4,73.0,3.9K,NaN
1678,1678,1678,1679,B. Monkey (1998),06-Feb-1998,NaN,https://www.imdb.com/title/tt0120594/,0,0,0,...,1,0,0,Alan Furnace is a young man with the perfectly...,Asia Argento|Jared Harris|Rupert Everett|Jonat...,Michael Radford,5.9,92.0,3.4K,NaN
1679,1679,1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,https://www.imdb.com/title/tt0120148/,0,0,0,...,0,0,0,"In London, the public relation Helen is fired ...",Gwyneth Paltrow|John Hannah|John Lynch|Jeanne ...,Peter Howitt,6.7,99.0,66K,NaN
1680,1680,1680,1681,You So Crazy (1994),01-Jan-1994,NaN,https://www.imdb.com/title/tt0111804/,0,0,0,...,0,0,0,"Stand up comedy by Martin Lawrence, filmed in ...",Martin Lawrence,Thomas Schlamme,6.6,84.0,952,NaN


# Extract Audio Features

In [ ]:
embeddings = {}
with tf.Graph().as_default(), tf.Session() as sess:
    # Define the model in inference mode, load the checkpoint, and
    # locate input and output tensors.
    vggish_slim.define_vggish_slim(training=False)
    vggish_slim.load_vggish_slim_checkpoint(sess, "./utils/vggish_model.ckpt")
    features_tensor = sess.graph.get_tensor_by_name(
        vggish_params.INPUT_TENSOR_NAME)
    embedding_tensor = sess.graph.get_tensor_by_name(
        vggish_params.OUTPUT_TENSOR_NAME)
    
    for index, movie in items.iterrows():
        try:
            examples_batch = vggish_input.wavfile_to_examples("../Datasets/ml-100k/Audio/{}.wav".format(movie['YT-Trailer ID']))
            [embedding_batch] = sess.run([embedding_tensor],
                                 feed_dict={features_tensor: examples_batch})
            if(movie['YT-Trailer ID'] == movie['YT-Trailer ID']):
                embeddings[movie['YT-Trailer ID']] = np.mean(embedding_batch, axis = 0)
                print(len(embeddings))
        except:
            pd.DataFrame(embeddings).to_csv("../Datasets/ml-100k/Audio/embeddings.csv")
            continue

INFO:tensorflow:Restoring parameters from ./utils/vggish_model.ckpt
1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
pd.DataFrame(embeddings).to_csv("../Datasets/ml-100k/Audio/embeddings.csv")